## Gerekli kütüphaneler:
Modelin kurulumu ve eğitim süreci için gerekli tüm paketleri içe aktarılır.

In [17]:
import sys

sys.path.append("../..")

import os
import cv2
import numpy as np
import joblib
from tqdm import tqdm

from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from utils.ml_features.preprocessing import extract_features, apply_pca
from utils.ml_features.grid_search import grid_search

## Veri yolu ve etiketler:
Veri setinin konumu `data_dir` değişkeninde tanımlanır ve klasör isimleri alfabetik olarak sıralanarak `class_names` listesi ile sınıf etiketleri elde edilir.

In [18]:
data_dir = "../../data/uc_merced"
class_names = sorted(os.listdir(data_dir))

## Feature çıkarma:
Her sınıfa ait tüm görüntüler okunur, 128×128 boyutuna getirilir ve RGB formatına dönüştürülür. `extract_features` fonksiyonu ile HOG, LBP, HSV histogramı ve GLCM tabanlı özellikler çıkarılarak `x_data` dizisine eklenir. Sınıf etiketleri `y_data` listesinde tutulur.

In [19]:
x_data, y_data = [], []

for label, cls in enumerate(class_names):
    cls_path = os.path.join(data_dir, cls)
    print("İşlenen class:", cls)
    for fname in tqdm(os.listdir(cls_path), desc=f"{cls}", leave=False):
        img_path = os.path.join(cls_path, fname)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (128, 128))
        features = extract_features(img)
        x_data.append(features)
        y_data.append(label)

x_data = np.array(x_data)
y_data = np.array(y_data)
print("Toplam örnek:", len(x_data), "| Feature boyutu:",  x_data.shape[1])

İşlenen class: agricultural


İşlenen class: airplane


İşlenen class: baseballdiamond


İşlenen class: beach


İşlenen class: buildings


...

İşlenen class: runway


İşlenen class: sparseresidential


İşlenen class: storagetanks


İşlenen class: tenniscourt


Toplam örnek: 2100 | Feature boyutu: 1803


## PCA kullanarak boyut azaltımı:
`apply_pca` fonksiyonu ile 256 bileşenli PCA uygulanarak özellik vektörlerinin boyutu küçültülür.

In [20]:
x_reduced = apply_pca(False, x_data, n_components=256)

## Veriyi bölme ve kaydetme:
PCA sonrası özellikler, %80 eğitim ve %20 test olacak şekilde `stratify` ile sınıf dengesi korunarak ayrılır. Elde edilen `x_train`, `y_train`, `x_test`, `y_test` verileri `.npy` formatında kaydedilir.

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x_reduced, y_data, test_size=0.2, stratify=y_data, random_state=42)

np.save("../../features/uc_merced_x_train.npy", x_train)
np.save("../../features/uc_merced_y_train.npy", y_train)
np.save("../../features/uc_merced_x_test.npy", x_test)
np.save("../../features/uc_merced_y_test.npy", y_test)

## Grid search ile random forest:
Random Forest modeli; ağaç sayısı `n_estimators`, maksimum derinlik `max_depth`,  
minimum bölünme örnek sayısı `min_samples_split`, özellik seçimi yöntemi `max_features` ve sınıf ağırlıkları  
`class_weight` hiperparametreleri ile 5 katlı çapraz doğrulama kullanılarak optimize edilir. En iyi model kaydedilir. `uc_merced_rf.pkl`

In [6]:
model = RandomForestClassifier(random_state=42)
params = {
    "n_estimators": [200, 300],
    "max_depth": [None, 25],
    "min_samples_split": [2, 5],
    "max_features": ["sqrt", 0.3],
    "class_weight": [None, "balanced"]
}

print("Random Forest eğitimi başladı.")
best_model, best_score, best_params, total_time = grid_search(model, params, x_train, y_train, cv=5)
joblib.dump(best_model, "../../models/ml_features/uc_merced_rf.pkl")
print("Random Forest için en iyi skor:", best_score, "| Parametreler:", best_params, "| Toplam Süre:", total_time, "\n")

Random Forest eğitimi başladı.
Denenecek hiperparametre kombinasyon sayısı: 32


RandomForestClassifier Grid Search:   3%|▎         | 1/32 [00:13<07:05, 13.72s/it]

Parametreler: {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 2, 'max_features': 'sqrt', 'class_weight': None} | Skor: 0.7107142857142857 | Süre: 13 sn


RandomForestClassifier Grid Search:   6%|▋         | 2/32 [00:26<06:32, 13.10s/it]

Parametreler: {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 2, 'max_features': 'sqrt', 'class_weight': 'balanced'} | Skor: 0.7107142857142857 | Süre: 12 sn


...

RandomForestClassifier Grid Search:  97%|█████████▋| 31/32 [17:14<00:41, 41.90s/it]

Parametreler: {'n_estimators': 300, 'max_depth': 25, 'min_samples_split': 5, 'max_features': 0.3, 'class_weight': None} | Skor: 0.7273809523809524 | Süre: 1 dk 4 sn


RandomForestClassifier Grid Search: 100%|██████████| 32/32 [18:19<00:00, 34.37s/it]

Parametreler: {'n_estimators': 300, 'max_depth': 25, 'min_samples_split': 5, 'max_features': 0.3, 'class_weight': 'balanced'} | Skor: 0.7273809523809524 | Süre: 1 dk 5 sn


Random Forest için en iyi skor: 0.7363095238095239 | Parametreler: {'n_estimators': 300, 'max_depth': None, 'min_samples_split': 5, 'max_features': 'sqrt', 'class_weight': None} | Toplam Süre: 18 dk 19 sn 



## Grid search ile HistGradientBoosting:
HistGradientBoosting modeli için öğrenme oranı `learning_rate`, maksimum derinlik `max_depth`,  
yaprak düğüm sayısı `max_leaf_nodes`, yaprak başına minimum örnek sayısı `min_samples_leaf`  
ve L2 düzenleme katsayısı `l2_regularization` hiperparametreleri 5 katlı çapraz doğrulama ile taranır.  
En iyi model kaydedilir. `uc_merced_hgb.pkl`

In [7]:
model = HistGradientBoostingClassifier(random_state=42)
params = {
    "learning_rate": [0.1, 0.2],
    "max_depth": [8, 12],
    "max_leaf_nodes": [63, 127],
    "min_samples_leaf": [20],
    "l2_regularization": [1.0]
}

print("HistGradientBoosting eğitimi başladı.")
best_model, best_score, best_params, total_time = grid_search(model, params, x_train, y_train, cv=5)
joblib.dump(best_model, "../../models/ml_features/uc_merced_hgb.pkl")
print("HistGradientBoosting  için en iyi skor:", best_score, "| Parametreler:", best_params, "| Toplam Süre:", total_time, "\n")

HistGradientBoosting eğitimi başladı.
Denenecek hiperparametre kombinasyon sayısı: 8


HistGradientBoostingClassifier Grid Search:  12%|█▎        | 1/8 [01:24<09:51, 84.48s/it]

Parametreler: {'learning_rate': 0.1, 'max_depth': 8, 'max_leaf_nodes': 63, 'min_samples_leaf': 20, 'l2_regularization': 1.0} | Skor: 0.7178571428571429 | Süre: 1 dk 24 sn


HistGradientBoostingClassifier Grid Search:  25%|██▌       | 2/8 [02:52<08:40, 86.78s/it]

Parametreler: {'learning_rate': 0.1, 'max_depth': 8, 'max_leaf_nodes': 127, 'min_samples_leaf': 20, 'l2_regularization': 1.0} | Skor: 0.7178571428571429 | Süre: 1 dk 28 sn


...

Parametreler: {'learning_rate': 0.2, 'max_depth': 8, 'max_leaf_nodes': 127, 'min_samples_leaf': 20, 'l2_regularization': 1.0} | Skor: 0.7214285714285714 | Süre: 1 dk 27 sn


HistGradientBoostingClassifier Grid Search:  88%|████████▊ | 7/8 [10:06<01:27, 87.09s/it]

Parametreler: {'learning_rate': 0.2, 'max_depth': 12, 'max_leaf_nodes': 63, 'min_samples_leaf': 20, 'l2_regularization': 1.0} | Skor: 0.7291666666666666 | Süre: 1 dk 27 sn


HistGradientBoostingClassifier Grid Search: 100%|██████████| 8/8 [11:34<00:00, 86.76s/it]


Parametreler: {'learning_rate': 0.2, 'max_depth': 12, 'max_leaf_nodes': 127, 'min_samples_leaf': 20, 'l2_regularization': 1.0} | Skor: 0.7291666666666666 | Süre: 1 dk 27 sn
HistGradientBoosting  için en iyi skor: 0.7291666666666666 | Parametreler: {'learning_rate': 0.2, 'max_depth': 12, 'max_leaf_nodes': 63, 'min_samples_leaf': 20, 'l2_regularization': 1.0} | Toplam Süre: 11 dk 34 sn 



## Grid search ile multilayer perceptron:
Veriler `StandardScaler` ile ölçeklendirilir. MLP modeli; gizli katman boyutları `hidden_layer_sizes`, düzenleme katsayısı `alpha`, öğrenme oranı `learning_rate_init`, çözücü `solver` ve aktivasyon fonksiyonu `activation` hiperparametreleri için 5 katlı çapraz doğrulama ile optimize edilir. En iyi model kaydedilir. `uc_merced_mlp.pkl`

In [23]:
model = MLPClassifier(
    max_iter=1500,
    early_stopping=True,
    n_iter_no_change=50,
    validation_fraction=0.25,
    random_state=42,
    tol=1e-6
)
params = {
    "hidden_layer_sizes": [
        (512,), (600,), (700,), (800,),
        (512, 128), (600, 150),
        (400, 400), (500, 500)
    ],
    "alpha": [1e-4, 3e-4, 5e-4, 7e-4],
    "learning_rate_init": [5e-4, 1e-3, 2e-3],
    "solver": ["lbfgs"],
    "activation": ["relu"]
}

print("Multilayer Perceptron eğitimi başladı.")
best_model, best_score, best_params, total_time = grid_search(model, params, x_train, y_train, cv=5)
joblib.dump(best_model, "../../models/ml_features/uc_merced_mlp.pkl")
print("Multilayer Perceptron için en iyi skor:", best_score, "| Parametreler:", best_params, "| Toplam Süre:", total_time, "\n")

Multilayer Perceptron eğitimi başladı.
Denenecek hiperparametre kombinasyon sayısı: 96


MLPClassifier Grid Search:   1%|          | 1/96 [00:02<04:01,  2.54s/it]

Parametreler: {'hidden_layer_sizes': (512,), 'alpha': 0.0001, 'learning_rate_init': 0.0005, 'solver': 'lbfgs', 'activation': 'relu'} | Skor: 0.5571428571428572 | Süre: 2 sn


MLPClassifier Grid Search:   2%|▏         | 2/96 [00:05<03:58,  2.54s/it]

Parametreler: {'hidden_layer_sizes': (512,), 'alpha': 0.0001, 'learning_rate_init': 0.001, 'solver': 'lbfgs', 'activation': 'relu'} | Skor: 0.5571428571428572 | Süre: 2 sn


...

MLPClassifier Grid Search:  99%|█████████▉| 95/96 [11:47<00:16, 16.92s/it]

Parametreler: {'hidden_layer_sizes': (500, 500), 'alpha': 0.0007, 'learning_rate_init': 0.001, 'solver': 'lbfgs', 'activation': 'relu'} | Skor: 0.5464285714285714 | Süre: 19 sn


MLPClassifier Grid Search: 100%|██████████| 96/96 [12:08<00:00,  7.59s/it]

Parametreler: {'hidden_layer_sizes': (500, 500), 'alpha': 0.0007, 'learning_rate_init': 0.002, 'solver': 'lbfgs', 'activation': 'relu'} | Skor: 0.5464285714285714 | Süre: 20 sn


Multilayer Perceptron için en iyi skor: 0.5744047619047619 | Parametreler: {'hidden_layer_sizes': (700,), 'alpha': 0.0005, 'learning_rate_init': 0.0005, 'solver': 'lbfgs', 'activation': 'relu'} | Toplam Süre: 12 dk 8 sn 

